In [1]:
import pandas as pd #librería para manejo de datos

datos= pd.read_csv("https://rtgodwin.com/data/houseprice.csv") #Lee base de datos de web...

In [2]:
import statsmodels.api as sm

y=datos["Price"]
X=datos[['Lot.Size', 'Age', 'Land.Value',   'Living.Area', 'Bedrooms']]

mco = sm.OLS(y, sm.add_constant(X)).fit()
print(mco.summary())


                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.614
Model:                            OLS   Adj. R-squared:                  0.612
Method:                 Least Squares   F-statistic:                     546.8
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        19:14:25   Log-Likelihood:                -21497.
No. Observations:                1728   AIC:                         4.301e+04
Df Residuals:                    1722   BIC:                         4.304e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        4.108e+04   6092.721      6.742      

# Golfeld-Quandt

In [11]:
import statsmodels.stats.diagnostic as sm_diagnostic
#GOLDFELD-QUANDT (Muestras Pequeñas)
GQ=sm_diagnostic.het_goldfeldquandt(y, sm.add_constant(datos["Age"]), alternative="increasing")
print("Goldfeld Quandt: ", GQ)

import statsmodels.stats.diagnostic as sm_diagnostic
#GOLDFELD-QUANDT (Muestras Pequeñas)
GQ=sm_diagnostic.het_goldfeldquandt(y, sm.add_constant(datos["Age"]), alternative="decreasing")
print("Goldfeld Quandt: ", GQ)

import statsmodels.stats.diagnostic as sm_diagnostic
#GOLDFELD-QUANDT (Muestras Pequeñas)
GQ=sm_diagnostic.het_goldfeldquandt(y, sm.add_constant(datos["Age"]), alternative="two-sided")
print("Goldfeld Quandt: ", GQ)

Goldfeld Quandt:  (0.9174070641302953, 0.8970472977178043, 'increasing')
Goldfeld Quandt:  (0.9174070641302953, 0.10295270228219566, 'decreasing')
Goldfeld Quandt:  (0.9174070641302953, 0.2059054045644021, 'two-sided')


# Breush-Pagan

In [5]:
#BREUSH-PAGAN
BP=sm_diagnostic.het_breuschpagan(mco.resid, mco.model.exog)
print("Breush Pagan: ", BP)

Breush Pagan:  (112.93359929175486, 9.824689989862927e-23, 24.08218731999146, 1.7388233505908317e-23)


# White

In [6]:
#WHITE
W=sm_diagnostic.het_white(mco.resid, mco.model.exog)
print("White: ", W)

White:  (136.58087964560715, 2.247089289312114e-19, 7.325020749503522, 2.8839807158113604e-20)


# Glejser

In [7]:
#GLEJSER
import numpy as np
z=np.array(datos["Land.Value"].values, dtype=float)
for h in [-2, -1, -0.5, 0.5, 1, 2]:
    # |e| = delta_0 + delta_1 z^h + eps
    mcoaux=sm.OLS(abs(mco.resid), sm.add_constant(z**h)).fit()
    pval=mcoaux.pvalues["x1"]
    print("h: ", h, "-> pvalt:", pval, "R2: ", mcoaux.rsquared)

h:  -2 -> pvalt: 0.7047749557327434 R2:  8.319103967269204e-05
h:  -1 -> pvalt: 0.08731978342281305 R2:  0.0016926273408465198
h:  -0.5 -> pvalt: 1.270199618513564e-06 R2:  0.013508522350844765
h:  0.5 -> pvalt: 2.5504307984884467e-29 R2:  0.0706196791733682
h:  1 -> pvalt: 4.091032734521836e-33 R2:  0.07991715587504711
h:  2 -> pvalt: 1.2193034771315243e-22 R2:  0.05405198733539229


$|e_i| = \alpha_0 + \alpha_1 z_i + v_i$

$w_i = \frac{1}{\sqrt{z_i}}$

# Mínimos Cuadrados Ponderados

In [9]:
mcp = sm.WLS(y, sm.add_constant(X), weights=1./np.sqrt(z)).fit()
print(mcp.summary())

                            WLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.593
Model:                            WLS   Adj. R-squared:                  0.592
Method:                 Least Squares   F-statistic:                     501.3
Date:                Tue, 19 Nov 2024   Prob (F-statistic):               0.00
Time:                        19:22:53   Log-Likelihood:                -21457.
No. Observations:                1728   AIC:                         4.293e+04
Df Residuals:                    1722   BIC:                         4.296e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          3.6e+04   5514.416      6.529      